In [1]:
from pennylane import numpy as np
import pennylane as qml
from pennylane import qchem
from matplotlib import pyplot as plt
#from functions_vqe_cnot import vqe_hee
#from functions_vqe_th import vqe_ng
from functions_vqe_crx_3 import vqe_ng
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## He<sub>2</sub> hamiltonian definition 

In [2]:
dataset = qml.data.load('qchem', molname="He2")[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  8
The Hamiltonian is  -1.6743100655776797 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.3416412951109973 * Z(0) + 0.01523407338776362 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.01523407338776362 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.3416408311620236 * Z(2) + 0.025441159889832372 * (Z(0) @ Z(2)) + 0.015234388911867857 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.015234388911867857 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 7.317214998671384e-08 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 7.317214998671384e-08 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.5193693204313432 * Z(4) + 0.09152839641299164 * (Z(0) @ Z(4)) + -0.5193733761290926 * Z(6) + 0.0915284611478574 * (Z(0) @ Z(6)) + 0.34164129511099733 * Z(1) + 0.14108387203893755 * (Z(0) @ Z(1)) + 0.03956129782205163 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.03956129782205163 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.11564281616296 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.11564281616296 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.11564281616296 * (X(0) @ X(1) @

In [3]:
dev = qml.device("lightning.qubit", wires=qubits)

In [4]:
electrons = 4
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qchem.excitations(electrons, qubits)
print(hf_state)

[1 1 1 1 0 0 0 0]


In [5]:
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

## Distribution of results

### Parameters

In [17]:
max_iterations = 200

layer = 24

n_runs = 30

conv_tol = 1e-06

weights_lr_list = np.linspace(0.1, 0.0001, 6)
phi_lr_list = np.linspace(0.1, 0.0001, 6)
n_weights = len(phi_lr_list)

theta = np.pi

In [18]:
prefix1 = "../local_data/he2_log2/"
spec1 = "_crx_1"

prefix2 = prefix1
spec2 = "_crx_2"

In [19]:
import json

run_info = {
    "max_iterations": max_iterations,
    "layers": layer,
    "n_runs": n_runs,
    "conv_tol": conv_tol,
    "weights_lr": weights_lr_list.tolist(),
    "phi_lr": phi_lr_list.tolist(),
    "prefix1": prefix1,
    "spec1": spec1,
    "prefix2": prefix2,
    "spec2": spec2,
    "device": device,
    "qubits": qubits,
    "thetas": theta
}

with open(prefix1 + "run_info.txt", "w") as f:
    json.dump(run_info, f, indent=4)

In [20]:
def run_single_simulation_2(i, k, j, weights_lr, phi_lr):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_ng(H, qubits, layer, theta=theta, weights_lr=weights_lr, phi_lr=phi_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)

    return i, k, j, (energy, grad_norm, grad_var, conv)

### NG

In [21]:
tasks = (delayed(run_single_simulation_2)(i, k, j, weights_lr, phi_lr)
         for i, weights_lr in enumerate(weights_lr_list)
         for k, phi_lr in enumerate(phi_lr_list)
         for j in range(n_runs))
print('number of tasks:', n_weights**2 * n_runs)
results = Parallel(n_jobs=20, verbose=11)(tasks)

number of tasks: 1080


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   26.3s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:   27.0s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:   30.4s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:   31.3s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:   31.4s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:   31.5s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:   36.4s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:   36.6s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:   36.8s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   37.4s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:   37.5s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:   37.5s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:   37.7s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:   38.2s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  

#### Save

In [22]:
results[0][3]

([-2.079606294631958,
  -2.079606294631958,
  -3.5305511951446533,
  -4.1365437507629395,
  -4.782752513885498,
  -5.0418548583984375,
  -5.117409706115723,
  -5.011110305786133,
  -5.031422138214111,
  -4.788000583648682,
  -4.9583892822265625,
  -4.710261344909668,
  -4.991162300109863,
  -4.817894458770752,
  -5.034373760223389,
  -4.909314155578613,
  -5.080706596374512,
  -4.987555503845215,
  -5.126015663146973,
  -5.054027557373047,
  -5.168927192687988,
  -5.112175941467285,
  -5.2083048820495605,
  -5.1635422706604,
  -5.243982315063477,
  -5.209053993225098,
  -5.276360988616943,
  -5.249497890472412,
  -5.305966854095459,
  -5.285628795623779,
  -5.333253383636475,
  -5.318115234375,
  -5.3585333824157715,
  -5.347482681274414,
  -5.381989002227783,
  -5.374115943908691,
  -5.403724670410156,
  -5.398301124572754,
  -5.423818111419678,
  -5.420271396636963,
  -5.442348480224609,
  -5.440232276916504,
  -5.45940637588501,
  -5.4583740234375,
  -5.475091457366943,
  -5.4748725

In [23]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_weights, n_weights, n_runs, max_iterations+1))
grad_norms = np.zeros((n_weights, n_weights, n_runs, max_iterations))
grad_variances = np.zeros((n_weights, n_weights, n_runs, max_iterations))
convergences = np.zeros((n_weights, n_weights, n_runs), dtype=bool)

for i, k, j, result in results:
    energies[i, k, j, :] = np.array(result[0])
    grad_norms[i, k, j, :] = np.array(result[1])
    grad_variances[i, k, j, :] = np.array(result[2])
    convergences[i, k, j] = result[3]

In [24]:
np.save(prefix2+"energies"+spec2+".npy", energies)
np.save(prefix2+"grad_norms"+spec2+".npy", grad_norms)
np.save(prefix2+"grad_variances"+spec2+".npy", grad_variances)
np.save(prefix2+"convergences"+spec2+".npy",convergences)